<a href="https://colab.research.google.com/github/sumanthd17/aspect-based-sentiment/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=61c749393df855e99e8ab611a7fdd07eecbd951ed89e09ed2d5f249961f0d7d2
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 161.0 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [2]:
!pip install transformers

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 1.1MB 17.1MB/s 
     |████████████████████████████████| 3.0MB 25.1MB/s 
     |████████████████████████████████| 890kB 35.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e092d2d7ae2b07f6168b7171c08423aebbb1805a069a80427d0989c55f918525
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import time
import datetime
import random
from tqdm import tqdm

import numpy as np
import pandas as pd

import nltk
nltk.download('punkt')

import torch
import torch.nn.functional as F
import transformers as optimus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
!git clone https://github.com/sumanthd17/aspect-based-sentiment.git

Cloning into 'aspect-based-sentiment'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 70 (delta 31), reused 50 (delta 19), pack-reused 0
Unpacking objects: 100% (70/70), done.


In [5]:
cd aspect-based-sentiment

/content/aspect-based-sentiment


In [6]:
def load_train_data(input_dir):
    df = pd.read_csv(input_dir + "train-QA.csv", sep="\t", names=['id', 'ques', 'ans', 'sentiment'])
    return df

In [7]:
def load_val_data(input_dir):
    df = pd.read_csv(input_dir + "val-QA.csv", sep="\t", names=['id', 'ques', 'ans', 'sentiment'])
    return df

In [8]:
def hyper_params():
    BATCH_SIZE = 32
    MAX_SEQ_LENGTH = 256
    LEARNING_RATE = 2e-5
    EPOCHS = 5
    WARMUP = 0.1
    return BATCH_SIZE, MAX_SEQ_LENGTH, LEARNING_RATE, WARMUP, EPOCHS

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
!python create_data.py

2977
747
1491


In [11]:
train_data = load_train_data('QA_pairs/')
val_data = load_val_data('QA_pairs/')

batch_size, max_seq_len, lr, warmup, epochs = hyper_params()
num_training_steps = int(len(train_data) / batch_size) * epochs
num_warmup_steps = warmup * num_training_steps
print(len(train_data))
print(len(val_data))

11256
2813


In [12]:
tokenizer_class, pretrained_weights = (
    optimus.BertTokenizer,
    "bert-base-uncased",
)

tokenizer = tokenizer_class.from_pretrained(pretrained_weights, do_lower_case=True)

In [13]:
sent2idx = {
    'None': 0,
    'Positive': 1,
    'Negative': 2
}

In [14]:
train = pd.DataFrame()

question_tokens = train_data['ques'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
q_segment_tokens = question_tokens.apply(lambda x: [0]*len(x))

answer_tokens = train_data['ans'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
answer_tokens = answer_tokens.apply(lambda x: [i for i in x if i != 101])
a_segment_tokens = answer_tokens.apply(lambda x: [1]*len(x))

train['input_ids'] = question_tokens + answer_tokens
train['segment_ids'] = q_segment_tokens + a_segment_tokens

train['input_mask'] = ''
for i, row in train.iterrows():
  train.iloc[i]['input_mask'] = [1]*len(train.iloc[i]['input_ids'])

train['input_mask'] = train['input_mask'].apply(lambda x: x + (max_seq_len - len(x))*[0])
train['input_ids'] = train['input_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])
train['segment_ids'] = train['segment_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])

train['label'] = ''
train['label'] = train_data['sentiment'].apply(lambda x: sent2idx[x])

In [15]:
val = pd.DataFrame()

question_tokens = val_data['ques'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
q_segment_tokens = question_tokens.apply(lambda x: [0]*len(x))

answer_tokens = val_data['ans'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
answer_tokens = answer_tokens.apply(lambda x: [i for i in x if i != 101])
a_segment_tokens = answer_tokens.apply(lambda x: [1]*len(x))

val['input_ids'] = question_tokens + answer_tokens
val['segment_ids'] = q_segment_tokens + a_segment_tokens

val['input_mask'] = ''
for i, row in val.iterrows():
  val.iloc[i]['input_mask'] = [1]*len(val.iloc[i]['input_ids'])

val['input_mask'] = val['input_mask'].apply(lambda x: x + (max_seq_len - len(x))*[0])
val['input_ids'] = val['input_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])
val['segment_ids'] = val['segment_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])

val['label'] = ''
val['label'] = train_data['sentiment'].apply(lambda x: sent2idx[x])

In [16]:
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.sampler import RandomSampler

In [17]:
input_ids = torch.tensor([r for r in train['input_ids']], dtype=torch.long)
input_masks = torch.tensor([r for r in train['input_mask']], dtype=torch.long)
label_ids = torch.tensor([r for r in train['label']], dtype=torch.long)
segment_ids = torch.tensor([r for r in train['segment_ids']], dtype=torch.long)

In [18]:
train_dataset = TensorDataset(input_ids, input_masks, segment_ids, label_ids)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

In [19]:
input_ids = torch.tensor([r for r in val['input_ids']], dtype=torch.long)
input_masks = torch.tensor([r for r in val['input_mask']], dtype=torch.long)
label_ids = torch.tensor([r for r in val['label']], dtype=torch.long)
segment_ids = torch.tensor([r for r in val['segment_ids']], dtype=torch.long)

In [20]:
val_dataset = TensorDataset(input_ids, input_masks, segment_ids, label_ids)
val_sampler = RandomSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size=batch_size)

In [21]:
model_class, pretrained_weights = (
    optimus.BertForSequenceClassification,
    "bert-base-uncased",
)

model = model_class.from_pretrained(pretrained_weights, num_labels=3)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [22]:
model = model.to(device)

In [23]:
optimizer_params = [
  {'params': [p for n, p in model.named_parameters()], 'weight_decay_rate': 0.0}
]

In [24]:
optimizer = optimus.AdamW(model.parameters(), lr=lr, correct_bias=False)
scheduler = optimus.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

In [25]:
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [26]:
training_stats = []

total_t0 = time.time()

for epoch_i in range(0, 5):
    ## TRAINING
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(tqdm(train_dataloader)):

        if step % 100 == 0 and not step == 0:
            elapsed = str(datetime.timedelta(seconds=int(round(time.time() - t0))))
            
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        input_ids, input_mask, segment_ids, label_ids = batch
  
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)

        model.zero_grad()        

        loss, _ = model(input_ids=input_ids,
                        attention_mask=input_mask,
                        token_type_ids=segment_ids,
                        labels=label_ids)

        total_train_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)          
    
    training_time = str(datetime.timedelta(seconds=int(round(time.time() - t0))))

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    ## VALIDATION

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for step, batch in enumerate(tqdm(val_dataloader)):
        
        input_ids, input_mask, segment_ids, label_ids = batch
  
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)
        
        with torch.no_grad():        
            loss, logits = model(input_ids=input_ids,
                        attention_mask=input_mask,
                        token_type_ids=segment_ids,
                        labels=label_ids)
            
        total_eval_loss += loss.item()

        logits = F.softmax(logits, dim=-1)
        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        outputs = np.argmax(logits, axis=1)

        total_eval_accuracy += np.sum(outputs == label_ids)
        

    avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(val_dataloader)
    
    validation_time = str(datetime.timedelta(seconds=int(round(time.time() - t0))))
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(str(datetime.timedelta(seconds=int(round(time.time() - total_t0))))))

  0%|          | 0/352 [00:00<?, ?it/s]

======== Epoch 1 / 5 ========
Training...


 28%|██▊       | 100/352 [04:11<10:28,  2.49s/it]

  Batch   100  of    352.    Elapsed: 0:04:11.


 57%|█████▋    | 200/352 [08:21<06:19,  2.49s/it]

...

  Batch   300  of    352.    Elapsed: 0:12:31.


  0%|          | 0/88 [00:00<?, ?it/s]


  Average training loss: 0.05
  Training epcoh took: 0:14:40

Running Validation...


100%|██████████| 88/88 [01:23<00:00,  1.06it/s]

  Accuracy: 21.88
  Validation Loss: 2.00
  Validation took: 0:01:23

Training complete!
Total training took 1:20:18 (h:mm:ss)


In [27]:
## INFERENCE

def load_test_data(input_dir):
    df = pd.read_csv(input_dir + "test-QA.csv", sep="\t", names=['id', 'ques', 'ans', 'sentiment'])
    return df

In [28]:
test_data = load_test_data('QA_pairs/')

In [29]:
idx2sentiment = {
    0: "None",
    1: "Positive",
    2: "Negative"
}

In [30]:
all_aspects = ['price',
               'shopping',
               'transit-location', 
               'safety',
               'nightlife',
               'live',
               'multiculture',
               'green-nature',
               'touristy',
               'quiet',
               'dining',
               'general']

In [31]:
test_grouped_by_id = test_data.groupby(['id'])

In [33]:
model.eval()

final_preds = pd.DataFrame()
test_accuracy = 0

for id, group in tqdm(test_grouped_by_id):
  test = pd.DataFrame()

  question_tokens = group['ques'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
  q_segment_tokens = question_tokens.apply(lambda x: [0]*len(x))

  answer_tokens = group['ans'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
  answer_tokens = answer_tokens.apply(lambda x: [i for i in x if i != 101])
  a_segment_tokens = answer_tokens.apply(lambda x: [1]*len(x))

  test['input_ids'] = question_tokens + answer_tokens
  test['segment_ids'] = q_segment_tokens + a_segment_tokens

  test['input_mask'] = test['input_ids'].apply(lambda x: [1]*len(x))

  test['input_mask'] = test['input_mask'].apply(lambda x: x + (max_seq_len - len(x))*[0])
  test['input_ids'] = test['input_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])
  test['segment_ids'] = test['segment_ids'].apply(lambda x: x + (max_seq_len - len(x))*[0])

  test['label'] = ''
  test['label'] = group['sentiment'].apply(lambda x: sent2idx[x])

  input_ids = torch.tensor([r for r in test['input_ids']], dtype=torch.long)
  input_mask = torch.tensor([r for r in test['input_mask']], dtype=torch.long)
  label_ids = torch.tensor([r for r in test['label']], dtype=torch.long)
  segment_ids = torch.tensor([r for r in test['segment_ids']], dtype=torch.long)

  input_ids = input_ids.to(device)
  input_mask = input_mask.to(device)
  segment_ids = segment_ids.to(device)
  label_ids = label_ids.to(device)

  with torch.no_grad():        
    loss, logits = model(input_ids=input_ids,
                        attention_mask=input_mask,
                        token_type_ids=segment_ids,
                        labels=label_ids)

  logits = F.softmax(logits, dim=-1)
  logits = logits.detach().cpu().numpy()
  label_ids = label_ids.to('cpu').numpy()
  outputs = np.argmax(logits, axis=1)

  test_accuracy += np.sum(outputs == label_ids)

  test['pred'] = outputs

  res = [idx for idx, val in enumerate(outputs) if val != 0]
  for val in res:
    d = {}
    d['id'] = id
    d['text'] = group.iloc[val]['ans']
    d['aspect'] = group.iloc[val]['ques'].split(' ')[6]
    d['sentiment'] = idx2sentiment[outputs[val]]
    d['target'] = 'LOCATION1' if 'LOCATION1' in group.iloc[val]['ans'] else 'LOCATION2'
    final_preds = final_preds.append(d, ignore_index=True)

print(test_accuracy/len(test_grouped_by_id))

100%|██████████| 1491/1491 [03:31<00:00,  7.05it/s]

3.5546613011401744


In [34]:
final_preds.to_csv('prediction.csv', index=False)

In [35]:
!nvidia-smi

Wed Jul 29 06:24:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    72W / 149W |  10833MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  